In [1]:
positive=open('positive-words.txt')
data=positive.read()
positive_words=data.split('\n')
positive_words.remove('')
positive.close()

In [2]:
negative=open('negative-words.txt')
data=negative.read()
negative_words=data.split('\n')
negative_words.remove('')
negative.close()

In [3]:
stop1=open('StopWords_Auditor.txt')
data=stop1.read()
stop_words1=data.split('\n')
stop_words1.remove('')
stop1.close()

In [4]:
stop2=open('StopWords_Currencies.txt')
data=stop2.read()
stop_words2=data.split('\n')
stop_words2.remove('')

In [5]:
stop_curr=[]
for x in stop_words2:
    a=x.split("|")
    stop_curr=stop_curr+a
stop2.close()

In [6]:
stop3=open('StopWords_DatesandNumbers.txt')
data=stop3.read()
stop_words3=data.split('\n')
stop3.close()

In [7]:
stop4=open('StopWords_Generic.txt')
data=stop4.read()
stop_words4=data.split('\n')
stop4.close()

In [8]:
stop5=open('StopWords_GenericLong.txt')
data=stop5.read()
stop_words5=data.split('\n')
stop5.close()

In [9]:
stop6=open('StopWords_Geographic.txt')
data=stop6.read()
stop_words6=data.split('\n')
stop_words6.remove('')
stop6.close()

In [10]:
stop7=open('StopWords_Names.txt')
data=stop7.read()
stop_words7=data.split('\n')
stop_words7.remove('')
stop7.close()

In [11]:
stopwords=stop_words1+stop_curr+stop_words3+stop_words4+stop_words5+stop_words6+stop_words7
len(stopwords)

14192

In [12]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
df=pd.read_csv('article.csv',encoding='ISO-8859-1')

,title,description
0,Rising IT cities and its impact on the economy...,We have seen a huge development and dependence...
1,Rising IT Cities and Their Impact on the Econo...,"Throughout history, from the industrial revolu..."
2,"Internet Demandâs Evolution, Communication I...","IntroductionIn the span of just a few decades,..."
3,Rise of Cybercrime and its Effect in upcoming ...,"The way we live, work, and communicate has unq..."
4,OTT platform and its impact on the entertainme...,The year 2040 is poised to witness a continued...
...,...,...
88,Estimating the impact of COVID-19 on the world...,COVID-19 an unprecedented pandemic for us but ...
89,Estimating the impact of COVID-19 on the world...,Whatâs perfection really? Does every person ...
90,Travel and Tourism Outlook,The UN projects a 20-30% decline in internatio...
91,Gaming Disorder and Effects of Gaming on Health.,Perhaps the virtual illusion has become todayâ...


In [14]:
port_stem=PorterStemmer()
def stemming(content):
    content=str(content)
    stemmed_content = content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
df['stemmed_content']=df['description'].apply(stemming)

In [23]:
def count_complex_words(words):
    a=(len(words)/(sum(1 for word in words if count_syllables(word) > 0)))
    return sum(1 for word in words if count_syllables(word) > 2),a
def count_syllables(word):
    word = word.lower()
    syllables = 0
    vowels = "aeiou"
    if word[0] in vowels:
        syllables += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            syllables += 1
    if word.endswith("es") or word.endswith("ed"):
        syllables -= 1
    if syllables == 0:
        syllables += 1
    return syllables
def average_word_length(words):
    total_chars = sum(len(word) for word in words)
    return total_chars / len(words)
avg_sen_len=[]
per_complex_word=[]
fog_index=[]
avg_no_words_per_sen=[]
com_word_count=[]
word_count=[]
syll_per_word=[]
personal_pronoun=[]
avg_word_count=[]

In [24]:
pos=[]
neg=[]
pol=[]
sub=[]
def preprocess_article(article):
    article=str(article)
    words = article.split()
    return words
def classify_article(article):
    words = preprocess_article(article)
    
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)
    polarity_score = (positive_count - negative_count) / ((positive_count + negative_count) + 0.000001)
    subjectivity_score = (positive_count + negative_count) / (len(words) + 0.000001)
    pos.append(positive_count)
    neg.append(negative_count)
    pol.append(polarity_score)
    sub.append(subjectivity_score)
    
    content=str(article)
    sentences = sent_tokenize(content)
    words = word_tokenize(content)
    avg_word_length=average_word_length(words)
    num_sentences = len(sentences)
    num_words = len(words)
    num_complex_words,syl_per_words = count_complex_words(words)
    pronouns = re.findall(r'\b(I|we|my|ours|us)\b', content, re.I)
    
    pronoun_len=len(pronouns)
    avg_sentence_length = int(num_words / num_sentences)
    percentage_complex_words = (num_complex_words / num_words)*100
    fo_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_sen_len.append(num_sentences)
    per_complex_word.append(percentage_complex_words)
    fog_index.append(fo_index)
    avg_no_words_per_sen.append(avg_sentence_length)
    com_word_count.append(num_complex_words)
    word_count.append(num_words)
    syll_per_word.append(syl_per_words)
    personal_pronoun.append(pronoun_len)
    avg_word_count.append(avg_word_length)
    
    if positive_count > negative_count:
        return "Positive"
    else:
        return "Negative"
df['sentiment']=df['description'].apply(classify_article)

In [25]:
df['positive_score']=pos
df['negative_score']=neg
df['Polarity_score']=pol
df['Subjectivity_score']=sub

In [26]:
df['Avg_sen_len']=avg_sen_len
df['per_com_word']=per_complex_word
df['fog_index']=fog_index
df['avg_wrod_per_sen']=avg_no_words_per_sen
df['com_words_count']=com_word_count
df['word_count']=word_count
df['syll_per_word']=syll_per_word
df['personal_pronoun']=personal_pronoun
df['Avg_word_count']=avg_word_count

In [28]:
df.to_csv("output.csv")